In [ ]:
#|default_exp testes

In [ ]:
#|export
import pytest
from validate_docbr import CPF, CNPJ
from receitaws.requisicoes import Requisicao


Validação de Parâmetros

In [ ]:
#|export
class TestParametros:
    @pytest.fixture
    def cpf_usuario(self):
        return CPF().generate()

    # Test case for valid arguments
    @pytest.mark.parametrize("tipo", ["cpf", "cnpj", "CPF", "CNPJ", "Cpf", "Cnpj"])
    @pytest.mark.parametrize("ambiente", ["ds", "hm", "pd"])
    @pytest.mark.parametrize("cache", [0, 3, 6, 9, 12, "3"])
    def test_valid_args(self, cpf_usuario, tipo, ambiente, cache):
        requisicao = Requisicao(
            cpf_usuario=cpf_usuario,
            tipo=tipo,
            origem="Teste",
            ambiente=ambiente,
            cache=cache,
        )
        assert requisicao.tipo == tipo.lower()
        if ambiente == "pd":
            ambiente = ""
        assert requisicao.ambiente == ambiente
        assert requisicao.cache == int(cache)

    # Test case for invalid tipo
    @pytest.mark.parametrize("tipo", ["invalid_tipo", None])
    @pytest.mark.parametrize("ambiente", ["ds", "hm", "pd"])
    def test_invalid_tipo(self, cpf_usuario, tipo, ambiente):
        with pytest.raises(AssertionError):
            requisicao = Requisicao(
                cpf_usuario, tipo, origem="Teste", ambiente=ambiente
            )

    # Test case for invalid origem
    @pytest.mark.parametrize("tipo", ["cpf", "cnpj"])
    @pytest.mark.parametrize("ambiente", ["ds", "hm", "pd"])
    def test_invalid_origem(self, cpf_usuario, tipo, ambiente):
        with pytest.raises(ValueError):
            Requisicao(cpf_usuario, tipo, origem="", ambiente=ambiente)

    # Test case for invalid ambiente
    @pytest.mark.parametrize("tipo", ["cpf", "cnpj"])
    def test_invalid_ambiente(self, cpf_usuario, tipo):
        with pytest.raises(AssertionError):
            Requisicao(cpf_usuario, tipo, "Teste", ambiente="Ambiente_Inválido")

    # Test case for invalid CPF
    @pytest.mark.parametrize("cpf", [9 * str(i) for i in range(10)])
    @pytest.mark.parametrize("tipo", ["cpf", "cnpj"])
    @pytest.mark.parametrize("ambiente", ["ds", "hm", "pd"])
    def test_invalid_cpf(self, cpf, tipo, ambiente):
        with pytest.raises(ValueError):
            Requisicao(cpf, tipo=tipo, origem="Teste", ambiente=ambiente)

    @pytest.mark.parametrize("tipo", ["cpf", "cnpj"])
    @pytest.mark.parametrize("ambiente", ["ds", "hm", "pd"])
    @pytest.mark.parametrize("cache", ["-1", -1])
    def test_invalid_cache(self, cpf_usuario, tipo, ambiente, cache):
        with pytest.raises(AssertionError):
            Requisicao(cpf_usuario, tipo, "Teste", ambiente, cache)